## Importing Libraries**

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 500)
pd.set_option("display.max_rows", 30)
pd.set_option("display.float_format", lambda x: '%.4f' % x)
# çıktının tek bir satırda olmasını sağlar.
pd.set_option("display.expand_frame_repr", False)
import datetime as dt

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Examining the Dataset

In [2]:
df_ = pd.read_csv("/kaggle/input/armut-dataset/armut_data.csv")
df = df_.copy()
df.head()

,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


**To see the associations, we create a new variable called 'Service'.

**

In [3]:
df["Service"] = df["ServiceId"].astype(str) + '_' + df["CategoryId"].astype(str)
df["new_date"] = pd.to_datetime(df["CreateDate"])
df["new_date"] = df["new_date"].dt.to_period('M')
df["Box_id"] = df["UserId"].astype(str) + '_' + df["new_date"].astype(str)

We create pivot table.

In [4]:
df_pivot = df.groupby(["Box_id", "Service"])["CreateDate"].count().unstack().fillna(0).applymap(lambda x: 1 if x>0 else 0)
df_pivot.head()

/tmp/ipykernel_17/1454961682.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_pivot = df.groupby(["Box_id", "Service"])["CreateDate"].count().unstack().fillna(0).applymap(lambda x: 1 if x>0 else 0)


Service,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,19_6,1_4,20_5,21_5,22_0,23_10,24_10,25_0,26_7,27_7,28_4,29_0,2_0,30_2,31_6,32_4,33_4,34_6,35_11,36_1,37_0,38_4,39_10,3_5,40_8,41_3,42_1,43_2,44_0,45_6,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
Box_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
0_2017-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
0_2018-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
0_2018-04,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10000_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


## Creating Association Rules


In [5]:
frequent_items = apriori(df_pivot,
                         min_support=0.005,
                         use_colnames=True)
frequent_items.sort_values("support", ascending=False)
rules = association_rules(frequent_items,
                          metric= "support",
                          min_threshold=0.005)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(22_0),(11_11),0.0475,0.0265,0.0083,0.1741,6.5623,0.0070,1.1786,0.8899
1,(11_11),(22_0),0.0265,0.0475,0.0083,0.3118,6.5623,0.0070,1.3840,0.8707
2,(25_0),(11_11),0.0429,0.0265,0.0074,0.1715,6.4668,0.0062,1.1750,0.8833
3,(11_11),(25_0),0.0265,0.0429,0.0074,0.2774,6.4668,0.0062,1.3245,0.8684
4,(2_0),(11_11),0.1303,0.0265,0.0079,0.0610,2.2998,0.0045,1.0367,0.6498


In [6]:
rules[(rules["support"] > 0.01) & (rules["confidence"] > 0.1) & (rules["lift"] > 2)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
26,(2_0),(15_1),0.1303,0.1210,0.0340,0.2606,2.1543,0.0182,1.1888,0.6161
27,(15_1),(2_0),0.1210,0.1303,0.0340,0.2807,2.1543,0.0182,1.2091,0.6095
28,(33_4),(15_1),0.0273,0.1210,0.0112,0.4113,3.4003,0.0079,1.4932,0.7257
56,(22_0),(25_0),0.0475,0.0429,0.0111,0.2340,5.4561,0.0091,1.2496,0.8575
57,(25_0),(22_0),0.0429,0.0475,0.0111,0.2592,5.4561,0.0091,1.2858,0.8533
60,(2_0),(22_0),0.1303,0.0475,0.0166,0.1272,2.6764,0.0104,1.0913,0.7202
61,(22_0),(2_0),0.0475,0.1303,0.0166,0.3487,2.6764,0.0104,1.3353,0.6576
66,(2_0),(25_0),0.1303,0.0429,0.0134,0.1031,2.4044,0.0078,1.0672,0.6716
67,(25_0),(2_0),0.0429,0.1303,0.0134,0.3133,2.4044,0.0078,1.2664,0.6103
82,(9_4),(38_4),0.0414,0.0666,0.0101,0.2432,3.6536,0.0073,1.2334,0.7577


**We rank services based on lift metric.

**

In [7]:
sorted_rules = rules.sort_values("lift", ascending=False)
Random_service = '2_0'

## Creating Recommandation List

We create recommendation function.



In [8]:
def arl_recommend(dataframe, Random_service, number_recommend):
    sorted_rules = rules.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, hizmet in enumerate(sorted_rules["antecedents"]):
        for j in hizmet:
            if list(hizmet)[0] == Random_service:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])
    return recommendation_list[0: number_recommend]

The 3 services we recommend to those who will buy '2_0 service'  are as follows.


In [9]:
arl_recommend(rules, '2_0', 3)

['22_0', '22_0', '25_0']